In [23]:
import numpy as np
import re
import nltk
from datasets import load_dataset
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid

import matplotlib.pyplot as plt

In [ ]:
# nltk.download('all')

# Part 0. Dataset Preparation

In [25]:
# loading the dataset from the library
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

In [26]:
# check the sizes of each dataset
train_size = len(train_dataset)
validation_size = len(validation_dataset)
test_size = len(test_dataset)

print(f"Training dataset size: {train_size}")
print(f"Validation dataset size: {validation_size}")
print(f"Test dataset size: {test_size}")

Training dataset size: 8530
Validation dataset size: 1066
Test dataset size: 1066


In [27]:
# view an example from each dataset
print("Train Dataset")
print(train_dataset.features)
print(train_dataset[0]) 

print("Test Dataset")
print(test_dataset.features)
print(test_dataset[0]) 

print("Validation Dataset")
print(validation_dataset.features)
print(validation_dataset[0])

Train Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
Test Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .', 'label': 1}
Validation Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children .', 'label': 1}


# Part 1. Preparing Word Embeddings

### Preprocessing

In [28]:
def preprocessing(text):

    # remove any other special characters but keep the general ones for potential sentiment usage
    text = re.sub(r'[^a-zA-Z0-9\'\!\?\.]', ' ', text)
    
    # replace multiple spaces with one space only
    text = re.sub(r'\s+', ' ', text)

    # remove leading and trailing whitespace to avoid unnecessary inconsistency 
    text = text.strip()

    return text

# apply the preprocessing function to the 'text' column of each dataset
train_dataset = train_dataset.map(lambda x: {'text': preprocessing(x['text'])})
validation_dataset = validation_dataset.map(lambda x: {'text': preprocessing(x['text'])})
test_dataset = test_dataset.map(lambda x: {'text': preprocessing(x['text'])})

# an example of the processed text
print("Train Dataset Example:")
print(train_dataset[0])

Train Dataset Example:
{'text': "the rock is destined to be the 21st century's new conan and that he's going to make a splash even greater than arnold schwarzenegger jean claud van damme or steven segal .", 'label': 1}


In [29]:
# tokenization
# empty list to store the resulting sentences
tokenized_sentences = []

for text in train_dataset['text']:
    # Tokenize the text and append the tokenized sentence to the list
    tokenized_sentences.append(word_tokenize(text))

### (a) Size of vocabulary in training data

In [30]:
# empty set for storing unique words
original_vocab = set()

for sentence in tokenized_sentences:
    for word in sentence:
        # add each word in the sentence to the words set
        original_vocab.add(word)

print(f"(a) The size of vocabulary formed in the training data is {len(original_vocab)}")

(a) The size of vocabulary formed in the training data is 16683


### (b) Number of OOV in the training data

In [31]:
# adjust the parameters for word2vec
vector_size = 100 # Dimensionality of the word vectors
window = 3 # Maximum distance between the current and predicted word within a sentence
min_count = 2 # Ignores all words with total frequency lower than this
workers = 4 # CPU cores
sg = 1 # 1 for skip-gram, 0 for CBOW
epochs = 5 

# train the word2vec model
word2vec_model = Word2Vec(
    sentences = tokenized_sentences, 
    vector_size = vector_size, 
    window = window, 
    min_count = min_count, 
    workers = workers,
    epochs = epochs)

# variable to store model's vocab list 
word2vec_vocab = set(word2vec_model.wv.key_to_index)

# Calculate OOV words by comparing the original vocab and Word2Vec vocab
oov_words = original_vocab - word2vec_vocab

print(f"(b) Number of OOV words in the training data is {len(oov_words)} when the minimum threshold for each word is {min_count}")

(b) Number of OOV words in the training data is 7866 when the minimum threshold for each word is 2


### (c) Mitigating OOV

In [32]:
# Replace all OOV words with <UNK>

# define the UNK and PAD token
UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'

# process each sentence in the tokenized_sentences list
for i, sentence in enumerate(tokenized_sentences):
    # empty list to store the current processed sentence
    processed_sentence = []
    for word in sentence:
        if word in word2vec_vocab:
            # if the current word is in the model's vocab, keep it as it is
            processed_sentence.append(word)  
        else:
            # otherwise, replace the word with UNK
            processed_sentence.append(UNK_TOKEN) 

    # update the sentence in the original tokenized_sentences list
    tokenized_sentences[i] = processed_sentence

## Mitigating OOV using FastText

In [33]:
from gensim.models import FastText

corpus = tokenized_sentences
fasttext_model = FastText(
    sentences=corpus, vector_size=100, window=3, min_count=2, workers=4, sg=1
)

fasttext_vocab = set(fasttext_model.wv.key_to_index)

ft_oov_words = original_vocab - fasttext_vocab

print(len(ft_oov_words))

7866


In [41]:
print(len(original_vocab))
print(original_vocab)

16683
{'mattei', 'ian', 'orc', 'dogged', 'petter', 'shimmering', 'stretch', 'captors', 'breadth', 'wim', 'psychopathy', 'daily', 'sanders', 'exact', 'stepmom', 'viciously', 'curiously', 'emerging', 'collegiate', 'chilled', 'injuries', 'adicional', 'kilted', 'desaponta', 'map', '104', 'scandalous', 'celebrated', 'tense', 'ellis', 'serrault', 'wrench', 'apollo', 'teenaged', 'goths', 'online', 'dani', 'investigation', 'dubbing', 'startle', 'directors', 'mixture', 'precise', 'mindedness', 'recorded', 'walker', 'warner', 'skittish', 'fingers', 'fessenden', 'hammy', 'flourishes', 'splat', 'sickening', 'igby', 'aim', 'transcendent', 'aplenty', 'rancid', 'buoys', 'imprimatur', 'extreme', 'perception', 'varmints', 'crumb', 'forced', 'material', 'ii', 'exercises', 'stanford', 'unable', 'threatens', 'blandness', 'preferable', 'skipping', 'offense', 'slow', 'inspired', 'rage', 'shootout', 'relying', 'bloodletting', 'antiseptic', 'worse', 'express', 'winners', 'harbor', 'colgate', 'unique', 'brave'

In [42]:
print(len(fasttext_vocab))
print(fasttext_vocab)

8818
{'mattei', 'ian', 'dogged', 'shimmering', 'breadth', 'daily', 'exact', 'curiously', 'emerging', 'chilled', 'map', 'celebrated', 'tense', 'ellis', 'serrault', 'wrench', 'apollo', 'directors', 'mixture', 'precise', 'warner', 'fingers', 'hammy', 'fessenden', 'flourishes', 'sickening', 'igby', 'aim', 'aplenty', 'rancid', 'extreme', 'perception', 'forced', 'material', 'ii', 'unable', 'threatens', 'blandness', 'slow', 'inspired', 'rage', 'relying', 'antiseptic', 'worse', 'express', 'winners', 'unique', 'brave', 'aquatic', 'lunacy', 'amuse', 'stake', 'screening', 'eventually', 'ria', 'malcolm', 'toe', 'incendiary', 'stew', 'agonizing', 'assess', 'old', 'misanthropic', 'leisurely', 'complicity', 'titus', 'haunting', 'ignore', 'canadians', 'returns', 'placing', 'sucked', 'meow', 'relocation', 'ironically', 'benefits', 'lively', 'smell', 'compared', 'matthew', 'violent', 'hey', 'worlds', 'uplifting', 'credit', 'herself', 'gentility', 'rings', 'embarrassment', 'beast', 'erotic', 'could', 'tu

In [43]:
print(ft_oov_words)

{'orc', 'petter', 'stretch', 'captors', 'wim', 'psychopathy', 'sanders', 'stepmom', 'viciously', 'collegiate', 'injuries', 'adicional', 'kilted', 'desaponta', '104', 'scandalous', 'goths', 'teenaged', 'online', 'dani', 'investigation', 'dubbing', 'startle', 'mindedness', 'recorded', 'walker', 'skittish', 'earmarks', 'splat', 'transcendent', 'buoys', 'imprimatur', 'varmints', 'crumb', 'exercises', 'stanford', 'skipping', 'preferable', 'offense', 'shootout', 'bloodletting', 'harbor', 'colgate', 'leukemia', 'ojos', 'manoel', 'jury', 'spiritedness', 'boiler', 'moralism', 'entrance', 'girlfriends', 'entretenimiento', 'unsung', 'temptation', 'extremists', 'cooly', 'peaked', 'roisterous', 'feisty', 'vim', 'agreeable', 'puppies', 'veljohnson', 'epis', 'subsided', 'bastards', 'crippled', 'returned', 'deliberateness', 'ostentatious', 'tumult', 'coinage', 'populates', 'amateurishness', 'morrison', 'clipped', 'improbability', 'schweiger', 'bullfighters', 'rapturous', 'backstory', 'sibling', 'desmo

In [44]:
print(len(word2vec_vocab))

8817


In [18]:
print(fasttext_model.wv["nonexistentword"])

[-0.16561644  0.11721427 -0.10187922 -0.10985807  0.03486895 -0.15660906
  0.16372027  0.36428457  0.06414182 -0.19543435  0.1193694  -0.01351071
  0.01828598  0.37551543 -0.13646527 -0.09924689  0.1908633  -0.1416742
 -0.32422826 -0.10793018 -0.1425223   0.08548637  0.11348457  0.07449151
 -0.09191715 -0.20784709 -0.21388939  0.09927446 -0.2979525   0.01422314
 -0.0674922   0.04552777  0.4139513  -0.02973791  0.06018949  0.1718147
  0.12381133  0.06572108 -0.16645296 -0.21976207  0.1596052  -0.00692129
 -0.09488826 -0.08375084 -0.0745609   0.01406283 -0.0987286   0.0322333
  0.24751003  0.28842393 -0.04110096  0.07403923 -0.0751178  -0.31984726
  0.05798929 -0.00319829  0.06222524 -0.08748318 -0.04123193 -0.04950598
 -0.16167806 -0.11492795  0.17886928  0.10838918  0.01413385  0.16821393
 -0.14462191 -0.01276385 -0.18015184 -0.14966294 -0.03732794  0.17121021
  0.1454438  -0.18676184 -0.01202812 -0.0052681   0.04059972  0.16207515
 -0.22240517  0.04976152 -0.07195684 -0.20886129 -0.13

### Embedding matrix

In [11]:
# empty set for storing unique words
final_vocab = set()

for sentence in tokenized_sentences:
    for word in sentence:
        # add each word in the sentence to the final_vocab set
        final_vocab.add(word)

# add 'UNK' and '<PAD>' to the vocabulary
final_vocab.add(UNK_TOKEN)
final_vocab.add(PAD_TOKEN)

# create the dictionary that maps each word in final_vocab to a unique index
word_to_index = {word: i for i, word in enumerate(final_vocab)}

embedding_dim = word2vec_model.vector_size 

# initialize embedding matrix with number of vocab and embedding dimension
embedding_matrix = np.zeros((len(word_to_index), embedding_dim))

# fill the embedding matrix with the corresponding word vectors
for word, i in word_to_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]
    else:
        # (option 1) random initialization for unknown words 
        embedding_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim,))
        # (option 2) use average vector for unknown words 
        # embedding_matrix[i] = np.mean(word2vec_model.wv.vectors, axis=0)

print(f"Shape of embedding matrix: {embedding_matrix.shape}")

Shape of embedding matrix: (8819, 100)


In [12]:
# convert word to indices 
def words_to_indices(sentence, word_to_index):
    return [word_to_index.get(word, word_to_index[UNK_TOKEN]) for word in sentence.split()]

train_X = [words_to_indices(sentence, word_to_index) for sentence in train_dataset['text']]
train_y = train_dataset['label']
val_X = [words_to_indices(sentence, word_to_index) for sentence in validation_dataset['text']]
val_y = validation_dataset['label']
test_X = [words_to_indices(sentence, word_to_index) for sentence in test_dataset['text']]
test_y = test_dataset['label']

def create_dataloader(X, y, batch_size=16, shuffle=True):
    X_tensor = [torch.tensor(seq, dtype=torch.long) for seq in X]
    X_padded = pad_sequence(X_tensor, batch_first=True, padding_value=word_to_index[PAD_TOKEN])
    y_tensor = torch.tensor(y, dtype=torch.long)
    dataset = TensorDataset(X_padded, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_dataloader = create_dataloader(train_X, train_y, shuffle=True)
val_dataloader = create_dataloader(val_X, val_y, shuffle=False)
test_dataloader = create_dataloader(test_X, test_y, shuffle=False)

# convert embedding_matrix to tensor
embedding_matrix = torch.FloatTensor(embedding_matrix)

# Part 3. Enhancement


## Part 3_1



In [20]:
class RNNUpdateEmbedding(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_dim,
        output_dim,
        pad_idx,
        embedding_matrix,
        freeze_embeddings=False,
        aggregation_method="max_pooling",
        dropout_rate=0.2,
    ):
        super().__init__()

        # convert numpy embedding matrix to tensor
        embedding_tensor = torch.FloatTensor(embedding_matrix)

        self.embedding = nn.Embedding.from_pretrained(
            embedding_tensor, padding_idx=pad_idx, freeze=freeze_embeddings
        )

        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        # attention layer for attention aggregation method
        self.attention = nn.Linear(hidden_dim, 1)

        self.aggregation_method = aggregation_method

    def forward(self, text):
        # embedded = self.embedding(text)
        embedded = self.dropout(self.embedding(text))
        output, hidden = self.rnn(embedded)

        if self.aggregation_method == "last_hidden":
            # use the last hidden state
            sentence_repr = hidden.squeeze(0)
        elif self.aggregation_method == "last_output":
            # use the last output
            sentence_repr = output[:, -1, :]
        elif self.aggregation_method == "mean_pooling":
            # average all outputs
            sentence_repr = torch.mean(output, dim=1)
        elif self.aggregation_method == "max_pooling":
            # max pooling over the sequence
            sentence_repr, _ = torch.max(output, dim=1)
        elif self.aggregation_method == "attention":
            # Attention mechanism
            attention_weights = F.softmax(self.attention(output), dim=1)
            sentence_repr = torch.sum(attention_weights * output, dim=1)
        sentence_repr = self.dropout(sentence_repr)
        return self.fc(sentence_repr)

In [21]:
vocab_size = len(word_to_index)
embedding_dim = embedding_matrix.shape[1]  # match word2vec vector size
hidden_dim = 256
output_dim = 2
pad_idx = word_to_index["<PAD>"] if "<PAD>" in word_to_index else 0

model = RNNUpdateEmbedding(
    vocab_size,
    embedding_dim,
    hidden_dim,
    output_dim,
    pad_idx,
    embedding_matrix,
    freeze_embeddings=False,
    aggregation_method="max_pooling",
)

# verify embedding layer
print(f"Embedding layer shape: {model.embedding.weight.shape}")
print(f"Embedding layer requires gradient: {model.embedding.weight.requires_grad}")

Embedding layer shape: torch.Size([8819, 100])
Embedding layer requires gradient: True


In [22]:
model.apply(lambda m: m.reset_parameters() if hasattr(m, "reset_parameters") else None)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"]
)

final_val_loss, final_val_acc = train_and_evaluate(
    model,
    train_dataloader,
    val_dataloader,
    criterion,
    optimizer,
    best_params["num_epochs"],
    best_params["patience"],
)

print(f"Final validation loss: {final_val_loss:.4f}")
print(f"Final validation accuracy: {final_val_acc:.4f}")

# Save the best model
torch.save(model.state_dict(), "./saved_models/part_3_1.pth")

Final validation loss: 0.5500
Final validation accuracy: 0.7213


In [23]:
best_params_enhanced_rnn, best_val_loss_enhanced_rnn, best_val_acc_enhanced_rnn = (
    hyperparameter_tuning(model, train_dataloader, val_dataloader)
)
print(f"Best parameters: {best_params_enhanced_rnn}")
print(f"Best validation loss: {best_val_loss_enhanced_rnn:.4f}")
print(f"Best validation accuracy: {best_val_acc_enhanced_rnn:.4f}")

# train using the best parameters
model.apply(lambda m: m.reset_parameters() if hasattr(m, "reset_parameters") else None)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=best_params_enhanced_rnn["lr"],
    weight_decay=best_params_enhanced_rnn["weight_decay"],
)

final_val_loss_enhanced_rnn, final_val_acc_enhanced_rnn = train_and_evaluate(
    model,
    train_dataloader,
    val_dataloader,
    criterion,
    optimizer,
    best_params_enhanced_rnn["num_epochs"],
    best_params_enhanced_rnn["patience"],
)

print(f"Final validation loss: {final_val_loss_enhanced_rnn:.4f}")
print(f"Final validation accuracy: {final_val_acc_enhanced_rnn:.4f}")

# Save the best model
torch.save(model.state_dict(), "./saved_models/part_3_1.pth")

Best parameters: {'batch_size': 16, 'lr': 0.001, 'num_epochs': 50, 'patience': 5, 'weight_decay': 1e-05}
Best validation loss: 0.5509
Best validation accuracy: 0.7095
Final validation loss: 0.5270
Final validation accuracy: 0.7442


In [24]:
model.load_state_dict(torch.load("./saved_models/part_3_1.pth"))

model.eval()
test_loss = 0
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, label in test_dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, label)

        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

test_loss /= len(test_dataloader)
test_acc = accuracy_score(all_labels, all_predictions)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

Test Loss: 0.5866
Test Accuracy: 0.7627


/var/folders/_j/9p9lzy_s3638t1nklml8vb400000gn/T/ipykernel_27956/1144306666.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./saved_mod